In [4]:
import os
import numpy as np
import moviepy.editor as mp
from googletrans import Translator
from gtts import gTTS
import subprocess
from TTS.api import TTS
import torch
from IPython.display import Audio, display
import whisper
import librosa
import soundfile as sf
from moviepy.editor import TextClip
from moviepy.editor import CompositeVideoClip
import soundfile as sf
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip
import re
from PIL import ImageFont

nombre_descargado = "video7.mp4"
ruta_carpeta_principal = "C://Users//R20//Desktop//videostraduccidos//"
ruta_carpeta_audios_esp = "audios_creados//audio_esp.wav"
ruta_carpeta_audios_ing = "audios_creados//audio_ing.wav"
ruta_carpeta_audios_fin = "audios_creados//audio_fin.wav"
ruta_carpeta_videofin = "videos_creados//" + nombre_descargado
nombre_video = "videos_descargados//" + nombre_descargado

rutaVideo = ruta_carpeta_principal + nombre_video
rutaAudioIngles = ruta_carpeta_principal + ruta_carpeta_audios_ing
rutaAudioGenerado =  ruta_carpeta_principal + ruta_carpeta_audios_esp
rutaAudioGeneradoNuevaVelocidad = ruta_carpeta_principal + ruta_carpeta_audios_fin
ruta_video_final = ruta_carpeta_principal + ruta_carpeta_videofin

target_language = "Japanese" 
# Mapping between full names and ISO 639-1 codes
language_mapping = {'English': 'en', 'Spanish': 'es', 'Chinese (Simplified)': 'zh-cn' , 'Japanese': 'ja' }

In [5]:
model = whisper.load_model("base")
result = model.transcribe(rutaVideo)

whisper_text = result["text"]
whisper_language = result['language']

print("Audio text:", whisper_text)

target_language_code = language_mapping[target_language]
from googletrans import Translator
translator = Translator()
translated_text = translator.translate(whisper_text, src=whisper_language, dest=target_language_code).text
print("Translated text:", translated_text)

Audio text:  1.500, es hora y media como máximo tu pueden ser lugar, nosotros vamos a...
Translated text: 1,500、それはあなたの最大の場所です、私たちは...


In [6]:
from google.cloud import texttospeech


text = translated_text  # Your Japanese text
language_code = "ja-JP"
ssml_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.WAV)
text_input = texttospeech.SynthesisInput(text=text)
voice = texttospeech.VoiceSelectionParams(
    name="ja-JP-Wavenet-A",  # Choose a Japanese voice
    language_code=language_code,
    ssml_gender=texttospeech.SsmlVoiceGender.FEMALE  # Optional: Set gender
)
client = texttospeech.TextToSpeechClient()
response = client.synthesize_speech(input=text_input, voice=voice, audio_config=ssml_config)
with open("output.mp3", "wb") as out:
    out.write(response.audio_content)

video = mp.VideoFileClip(rutaVideo)
audio_clip = video.audio
audio_clip.write_audiofile(rutaAudioIngles)


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
translated_text = translated_text.replace('.', '. ').replace('?', '? ')

cambio_velocidad = 1.1
velocidad_correcta = False
y_original, sr_original = librosa.load(rutaAudioIngles)
y_generated, sr_generated = librosa.load(rutaAudioGenerado)
duration_original = len(y_original) / sr_original
duration_original_esp = len(y_generated) / sr_generated
print(f"La duración del audio original es de {duration_original:.2f} segundos")
y_adjusted = y_original
resta_originales = duration_original - duration_original_esp

speed_ratio = cambio_velocidad 
sr_adjusted = int(sr_original * speed_ratio) 
y_adjusted = librosa.effects.time_stretch(y_generated, rate=speed_ratio)
duration_adjusted = len(y_adjusted) / sr_adjusted
resta = duration_original - duration_adjusted
display(resta)

sf.write(rutaAudioGeneradoNuevaVelocidad, y_adjusted, sr_original)
# Mostrar el audio
audio_widget = Audio(filename=rutaAudioGeneradoNuevaVelocidad, autoplay=False)
display(audio_widget)

In [ ]:
video = mp.VideoFileClip(rutaVideo)

# Cargar el audio en español
audio_spanish = mp.AudioFileClip(rutaAudioGeneradoNuevaVelocidad)

# Set the audio of the composite clip
video.audio = audio_spanish

# Save the resulting clip
video.write_videofile(ruta_video_final)